In [1]:
from vpython import *
import pandas as pd
import numpy as np
import random
import os
import imageio
import pathlib
import pandas as pd
import time
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
limegreen = vector(0.19607843137254902, 0.803921568627451, 0.19607843137254902)
dodgerblue = vector(0.11764705882352941, 0.5647058823529412, 1)

<IPython.core.display.Javascript object>

# Single Run

In [2]:
def single_run(run_no, root="F:\\Simulation", r_min=1, lifetime=5, gif = True, frames_for_gif=500):
    """
    Shows an animation of the simulation within the notebook. If gif==True, every frames_for_gif frames are saved as separate png files (in the Downloads directory)
    Parameters:
    - run_no (int): The run number of the simulation you'd like to animate.
    - root (str): The root directory path for the simulation output files. Default is "F:\\Simulation".
    - r_min (int): The minimum radius used in the simulation. Default is 1.
    - lifetime (int): The lifetime of the cells. Default is 5. Used to define cell radius.
    - gif (bool): If True, saves frames from the animation to be turned into a gif. Default is True.
    - frames_for_gif (int): The frame number to be saved into a gif. Default is every 500th frame.

    """
    positions = np.load(root+".\\Alterall_positions_{}.npy".format(run_no), allow_pickle=True)
    ages = np.load(root+".\\Alterall_ages_{}.npy".format(run_no), allow_pickle=True)
    parameters = pd.read_parquet(root+".\\AlterALL_Run{}.parquet".format(run_no), engine='pyarrow').iloc[0][["beta", "a_eq_star_scaling", "lumen_radius_scaling"]]
    my_title = "beta={}, b={}, f = {}".format(round(parameters.beta,2), round(parameters.a_eq_star_scaling,2),round(parameters.lumen_radius_scaling, 2))
    for i in range(1, 400):
        globals()[f'cell{i}'] = 1

    # Instantiate 400 objects and store out of frame until needed
    cells = [globals()[f'cell{i}'] for i in range(1, 400)]
    for i in range(len(cells)):
        cells[i]= sphere(color = limegreen, radius = 0.3, pos=vector(50,50,50))

    max_positions = [position.max(axis=0) for position in positions]
    min_positions = [position.min(axis=0) for position in positions]
    max_x = max([position[0] for position in max_positions])
    max_y = max([position[0] for position in max_positions])
    max_z = max([position[0] for position in max_positions])

    min_x = min([position[0] for position in min_positions])
    min_y = min([position[0] for position in min_positions])
    min_z = min([position[0] for position in min_positions])

    max_positions = np.array((max_x, max_y, max_z))
    min_positions = np.array((min_x, min_y, min_z))
    center = ((max_positions + min_positions)/2)
    scene.center = vector(center[0], center[1],center[2])
    my_range = (max_positions - min_positions).max()
    scene.range = my_range + 2    
    scene.resizable = False       # turns off user resizing
    scene.userzoom = False 

    T = label(text=my_title, pos=vector(center[0], center[1],center[2]), box=False, line=False, yoffset = 120)
    for j, position in enumerate (positions):
        rate(100000)     
        N_cells = position.shape[0]
        for i, cell in enumerate(cells[:N_cells]):
            
            if i == N_cells-1:
                cell.color = limegreen
            else:
                cell.color = dodgerblue
            cell.pos = vector(position[i,0], position[i,1], position[i,2]) 
            cell.radius = r_min * (1 + ((np.cbrt(2)-1)*ages[j][i]*(1/lifetime)))
        for i, cell in enumerate(cells[N_cells:]):
            cell.pos = vector(50,50,50)
            cell.color = color.red
        if gif == True:
            if j%frames_for_gif ==0:
                scene.capture("{}_Run_{}".format(int(j/frames_for_gif),run_no))

single_run(214)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Multiple Runs

In [2]:
def final_shot_of_multiple(all_i, root = "F:\\Simulation", title=True, r_min=1, lifetime=5):
    """
    Saves a .png file of the final frame of each simulation in all_i (default storage location is Downloads). 
    Parameters:
    - all_i (list): A list of the run numbers of the simulations you'd like to animate.
    - title(Bool): If title==True, the simulation parameters are displayed as a title.
    - root (str): The root directory path for the simulation output files. Default is "F:\\Simulation".
    - r_min (int): The minimum radius used in the simulation. Default is 1.
    - lifetime (int): The lifetime of the cells. Default is 5. Used to define cell radius.
    """
    all_ages = []
    all_positions = []
    all_run_data = []
    for i in all_i:
        all_ages.append(root+".\\AlterALL_ages_{}.npy".format(i))
        all_positions.append(root+".\\AlterALL_positions_{}.npy".format(i))
        all_run_data.append(root+".\\AlterALL_Run{}.parquet".format(i))
    cells = [sphere(color=limegreen, radius=0.3, pos=vector(50,50,50)) for _ in range(400)]
    if title ==True:
        T = label(text="X", pos=vector(0,0,0), box=False, line=False)

    for i, position in enumerate(all_positions):
        rate(21)
        try:
            final_positions = np.load(position, allow_pickle=True)[-1]
            final_ages = np.load(all_ages[i], allow_pickle=True)[-1]  
            max_positions = final_positions.max(axis = 0)
            min_positions = final_positions.min(axis = 0)
            center = ((max_positions + min_positions)/2)
            scene.center = vector(center[0], center[1],center[2])
            my_range = (max_positions - min_positions).max()
            scene.range = my_range + 2
            N_cells = final_positions.shape[0]
            for j, cell in enumerate(cells[:N_cells]):
                if j == N_cells-1:
                    cell.color = limegreen
                else:
                    cell.color = dodgerblue
                cell.pos = vector(final_positions[j,0], final_positions[j,1], final_positions[j,2]) 
                cell.radius = r_min * (1 + ((np.cbrt(2)-1)*final_ages[j]*(1/lifetime)))
            for cell in (cells[N_cells:]):
                cell.pos = vector(50,50,50) 
            if title == True:
                parameters = pd.read_parquet(root+".\\AlterALL_Run{}.parquet".format(i), engine='pyarrow').iloc[0][["beta", "a_eq_star_scaling", "lumen_radius_scaling"]]
                my_title = "beta={}, b={}, f = {}".format(round(parameters.beta,2), round(parameters.a_eq_star_scaling,2),round(parameters.lumen_radius_scaling, 2))
                T.text = my_title
                T.pos = vector(center[0], center[1], center[2])
                T.yoffset = 120
            scene.capture("Run_{}".format(i))
        except:
            print(i)
            continue
final_shot_of_multiple([i for i in range(5)], title=True)

# Create Gif

In [5]:
def create_gif(run_no, directory = 'C:\\Users\\Bel\\Downloads', max_no_frames=300, duration=15):
    """
    Creates a gif from the png files saved in the Downloads directory. 
    Parameters:
    - directory (str): The directory path where the png files are stored. Default is 'C:\\Users\\Bel\\Downloads'.
    - run_no (int): The run number of the simulation you'd like to animate. Default is 0.
    - max_no_frames (int): The maximum number of frames to be included in the gif. Default is 300.
    - duration (int): The duration of each frame in the gif in seconds. Default is 15.
    """
    images = []
    for i in range(max_no_frames):
        try:
            images.append(imageio.imread(directory + "\\{}_Run_{}.png".format(i, run_no)))
        except:
            pass
    output_file = os.path.join(directory, 'Run{}_output_{}s.gif'.format(run_no, duration))
    imageio.mimsave(output_file, images, duration=duration/60)

C:\Users\isobe\AppData\Local\Temp\ipykernel_25064\1418501459.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))
